# Simulasi Perhitungan PCA Manual 

## Memuat Library

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Membuat Dataframe 2x2 dari Dataset Keseluruhan

In [ ]:
# Definisikan data sebagai dictionary dengan dua key: 'reg_gdp' dan 'life_exp'
data = { 

  # 'reg_gdp' adalah list yang berisi nilai-nilai GDP regional
  'reg_gdp': [2.275, 2.425, 5.531, 5.063, 10.616], 
  
  # 'life_exp' adalah list yang berisi nilai-nilai harapan hidup
  'life_exp': [65.240, 67.355, 64.360, 68.155, 68.705] 
}


In [ ]:
# Buat DataFrame dari dictionary data
sc = pd.DataFrame(data)

# Tampilkan DataFrame sc
sc

,reg_gdp,life_exp
0,2.275,65.240
1,2.425,67.355
2,5.531,64.360
3,5.063,68.155
4,10.616,68.705


## Statistik Deskriptif dari Dataframe

### Rata-Rata

In [55]:
sc.mean()

reg_gdp      5.182
life_exp    66.763
dtype: float64

#### Simpangan Baku



- sc.std() adalah fungsi untuk menghitung standar deviasi.
- ddof=0 adalah parameter yang menentukan derajat kebebasan (delta degrees of freedom). 

Dalam hal ini, ddof=0 berarti standar deviasi dihitung dengan membagi jumlah kuadrat deviasi dengan jumlah total data (N), bukan dengan N-1.


In [56]:
sc.std(ddof=0)

reg_gdp     3.023773
life_exp    1.682464
dtype: float64

## Normalisasi

### Standar Scaler (Z-Score Normalization)


$$
Z = \frac{X - \mu}{\sigma}
$$

di mana:

* $X$: nilai asli
* $\mu$: mean fitur
* $\sigma$: standar deviasi fitur

In [ ]:
# Buat objek StandardScaler
scaler = StandardScaler()

# Standardisasi data menggunakan fit_transform
sc_s = scaler.fit_transform(sc)

# Tampilkan data yang telah distandardisasi
sc_s

array([[-0.96138169, -0.90521982],
       [-0.91177479,  0.35186483],
       [ 0.11541872, -1.42826213],
       [-0.03935481,  0.82735784],
       [ 1.79709257,  1.15425929]])

## Covarians Matrix

### Menghitung Matriks Kovarians

$$
\text{Cov}(X) = \frac{1}{n-1} X^T X
$$

In [ ]:
# Menghitung matriks kovarians dari data yang telah distandardisasi
covmat_sc = np.cov(sc_s, rowvar=False)

# Tampilkan matriks kovarians
print("Matriks Kovarians: ")
print(covmat_sc)

Matriks Kovarians: 
[[1.25       0.60658559]
 [0.60658559 1.25      ]]


### Mengubah Matriks Kovarians Menjadi Data Frame

In [ ]:
# Membuat DataFrame dari matriks kovarians
cov_df = pd.DataFrame(covmat_sc, columns=sc.columns, index=sc.columns)

# Menampilkan DataFrame
print("\nDataFrame Matriks Kovarians:")
cov_df


DataFrame Matriks Kovarians:


,reg_gdp,life_exp
reg_gdp,1.250000,0.606586
life_exp,0.606586,1.250000


## Eigen Values (λ) dan Eigen Vectors (kolom adalah vektor eigen)

$$
\text{Cov}(X) \cdot v = \lambda \cdot v
$$

### Menghitung Eigen Values dan Eigen Vectors

In [60]:
# Menghitung Eigenvalues dan Eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(covmat_sc)

# Membuat DataFrame untuk Eigenvalues
eigenvalue_df = pd.DataFrame({'Eigenvalue': eigenvalues})

# Menampilkan DataFrame Eigenvalues
print("\nEigenvalues:")
eigenvalue_df


Eigenvalues:


,Eigenvalue
0,1.856586
1,0.643414


### Membuat Data Frame untuk Eigen Vectors

In [61]:
# Membuat DataFrame untuk Eigenvectors
eigenvector_df = pd.DataFrame(eigenvectors, index=sc.columns)

# Menampilkan DataFrame Eigenvectors
print("\nEigenvectors:")
eigenvector_df


Eigenvectors:


,0,1
reg_gdp,0.707107,-0.707107
life_exp,0.707107,0.707107


### Menghitung Varians Kumulatif

Kode berikut digunakan untuk menghitung varians kumulatif dari eigenvalues yang telah diurutkan secara descending.

- var_exp adalah daftar yang berisi proporsi varians yang dijelaskan oleh setiap komponen utama (PC), dihitung dengan membagi setiap eigenvalue dengan jumlah total eigenvalues.
- cum_var_exp adalah array yang berisi varians kumulatif, dihitung dengan menggunakan fungsi np.cumsum() untuk mengakumulasi proporsi varians yang dijelaskan oleh setiap PC.

Dengan mencetak cum_var_exp, kita dapat melihat seberapa besar proporsi varians yang dijelaskan oleh setiap komponen utama secara kumulatif.

In [ ]:
# Menghitung proporsi varians yang dijelaskan oleh setiap komponen utama
var_exp = [(i / sum(eigenvalues)) 
           for i in sorted(eigenvalues, reverse=True)]

# Menghitung varians kumulatif
cum_var_exp = np.cumsum(var_exp)

# Menampilkan varians kumulatif
print("Varians Kumulatif")
print(cum_var_exp)

Varians Kumulatif
[0.74263424 1.        ]


### Menentukan Jumlah Komponen Utama

Kode berikut digunakan untuk menentukan jumlah komponen utama yang dipilih berdasarkan pada proporsi varians kumulatif yang diinginkan (dalam hal ini 80%).

- np.argmax(cum_var_exp <= 0.80) mencari indeks pertama dari cum_var_exp yang memiliki nilai lebih besar dari 0.80. Namun, karena np.argmax mengembalikan indeks dari nilai pertama yang memenuhi kondisi, maka kita perlu menambahkan 1 untuk mendapatkan jumlah komponen utama yang sebenarnya.
- n_components adalah jumlah komponen utama yang dipilih.

Dengan demikian, kode tersebut akan memilih jumlah komponen utama yang diperlukan untuk menjelaskan setidaknya 80% dari varians total dalam data.


In [ ]:
# Menentukan jumlah komponen utama yang dipilih
n_components = np.argmax(cum_var_exp >= 0.80) + 1

# Menampilkan jumlah komponen utama yang dipilih
print(f"Jumlah komponen utama yang dipilih: {n_components}")

Jumlah komponen utama yang dipilih: 1


## PCA

### Menghitung PCA

In [ ]:
# Membuat objek PCA dengan jumlah komponen utama yang telah ditentukan
pca = PCA(n_components=n_components)

# Melakukan PCA pada data yang telah distandardisasi
X_pca = pca.fit_transform(sc_s)

### Mengambil Eigenvalues dan Eigenvectors dari PCA

Kode berikut digunakan untuk mengambil eigenvalues dan eigenvectors dari objek PCA (Principal Component Analysis).

- pca.explained_variance_ adalah atribut yang menyimpan eigenvalues dari PCA, yang merepresentasikan jumlah varians yang dijelaskan oleh setiap komponen utama.
- pca.components_ adalah atribut yang menyimpan eigenvectors dari PCA, yang merepresentasikan arah dari setiap komponen utama.

Dengan mengambil eigenvalues dan eigenvectors, dapat dilakukan analisis lebih lanjut, seperti:

- Menghitung proporsi varians yang dijelaskan oleh setiap komponen utama
- Menghitung varians kumulatif
- Memilih jumlah komponen utama yang optimal
- Menganalisis struktur data menggunakan eigenvectors

In [ ]:
# Mengambil eigenvalues dan eigenvectors
eigenvalues = pca.explained_variance_

# Mengambil eigenvalues dan eigenvectors
eigenvalues = pca.explained_variance_
eigenvectors = pca.components_

### Menampilkan Eigen Vectors

In [66]:
print(eigenvectors)

[[0.70710678 0.70710678]]


### Mengubah Eigen Vectors Menjadi Data Frame

In [67]:
eigenvector_df = pd.DataFrame(eigenvectors, columns=sc.columns)
eigenvector_df

,reg_gdp,life_exp
0,0.707107,0.707107


### Membuat Data Frame Skor Komponen Utama


Kode berikut digunakan untuk membuat DataFrame yang berisi skor komponen utama dari data yang telah ditransformasikan menggunakan PCA.

- X_pca adalah array yang berisi skor komponen utama, yang diperoleh dari transformasi data asli menggunakan PCA.
- df_scores adalah DataFrame yang berisi skor komponen utama, dengan kolom yang diberi label "PC1", "PC2", ..., "PCn" sesuai dengan jumlah komponen utama yang dipilih.

Dengan membuat DataFrame skor komponen utama, kita dapat:

- Menganalisis struktur data yang telah direduksi dimensinya
- Menggunakan skor komponen utama sebagai input untuk analisis lanjutan, seperti clustering atau klasifikasi
- Memvisualisasikan data menggunakan skor komponen utama.

In [ ]:
# Skor Komponen Utama
df_scores = pd.DataFrame(X_pca, columns=[f"PC{i}" for i in range(1, n_components + 1)])
print(df_scores)

        PC1
0 -1.319887
1 -0.395916
2 -0.928320
3  0.557202
4  2.086921
